# Welcome to NeuCoReClass AD 🚀

This notebook is a friendly introduction to using NeuCoReClass-AD for time series anomaly detection. Here, you will learn step by step how to:

📂 Load your data

⚙️ Set the hyperparameters of the method

⚡ Launch experiments and evaluate results

The goal is to give you a quick and practical walkthrough so you can easily apply NeuCoReClass AD to any dataset.

👉 It's very simple, so let's get started!

## Load Resources 🛠️

The code required to use NeuCoReClass-AD can be imported from the following modules and scripts:

In [1]:
from models import *
from datasets import *
from utils import *

## Load Data 💾

For the experiments, we will use datasets from the UCR Time Series Classification Archive.
Data loading is handled with the aeon library through a helper function we provide, called get_dataset().

When loading a dataset, you will need to specify:

- dataset: the name of the dataset to load

- normal: which class is considered the normal one

- reverse: defines how anomalies are assigned. Two possible values:

    * False: the class defined in normal is treated as normal, and all others are anomalies

    * True: all classes except the one defined in normal are treated as normal, and that class is considered anomalous

The function get_dataset() returns:

- dataset: name of the dataset loaded

- train_data: training samples (only from the normal class)

- test_data: test samples (all classes included)

- test_labels: binary labels for evaluation (1 = normal, 0 = anomalous)

- real_labels: the original class names of the test samples

👉 Additionally, the function also prints information about the existing classes in the dataset, clearly indicating which ones have been defined as normal and which as abnormal.

In [2]:
dataset, train_data, test_data, test_labels, real_labels  = get_dataset(dataset="Epilepsy", normal="sawing", reverse=False)

Problem name:  epilepsy
Existing Classes:  ['epilepsy', 'walking', 'running', 'sawing']
Normal Class(es):  ['sawing']
Anomalous Class(es):  ['epilepsy' 'running' 'walking']


Now we split 90% of the training data for actual training, and keep the remaining 10% as a validation set (to help prevent overfitting). This way, the model can be trained on the majority of the normal samples while still having a small subset for validation and hyperparameter tuning.

In [3]:
train_size = int(0.9 * len(train_data))
val_size = len(train_data) - train_size
train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

## Instantiate, train and evaluate NeuCoReClass AD

The next step is to instantiate the NeuCoReClass-AD model, train it on the normal data, and finally evaluate its performance on the test set. 

Experiments of NeuCoReClass AD rely on a set of hyperparameters that control both the training process and the behavior of the model:

- n_transforms: number of neural transformations applied (default: 12)

- measure: similarity measure used for contrastive learning (default: "cosine")

- temperature: scaling factor for the contrastive loss (default: 0.1)

- batch_size: size of the mini-batches during training (default: 32)


The NeuCoReClass-AD object also requires additional parameters when being instantiated, including:

- input_dims: the number of features in the original time series

- train_data: the partition of normal data used for training

- val_data: the partition of normal data used for validation

- test_data: the data used for evaluation (including both normal and anomalous samples)

In [5]:
#  We will use the default values for the hyperparameters of NeuCoReClass AD.

input_dims = train_data[0].shape[1]
model = Experiment(input_dims = input_dims, 
                   train_data = train_dataset, 
                   val_data = val_dataset, 
                   test_data = test_data, 
                   n_transforms = 12, 
                   temperature = 0.1, 
                   batch_size = 32, 
                   measure = "cosine").to(device)


Then, we train the model. For this step, we need to fix two key hyperparameters:

- max_epochs: maximum number of training epochs (default: 1000)
- verbose: whether to print detailed training logs (default: False)

This function returns a list denoting the evolution of the training loss

In [7]:
# We set the number of epochs as 
loss_log = model.train_model(max_epochs=1, verbose=True)

Epoch #0: loss=4.031190872192383, val_loss=3.9424713452657065


Finally, we evaluate the model on the evaluation set, computing two standard metrics for anomaly detection:

- AUROC (Area Under the Receiver Operating Characteristic Curve)

- AUPR (Area Under the Precision–Recall Curve)

These metrics provide a reliable assessment of the model’s ability to distinguish between normal and anomalous sequences.

In [8]:
model.eval()
scores = model.compute_scores()
auroc, aupr = metrics(scores, test_labels)

print("AUROC: ", auroc)
print("AUPR: ", aupr)

AUROC:  0.8817901234567901
AUPR:  0.9683922358554421


# One-Click Experiments with run_experiment() ⚡

If you prefer not to go through each step manually, NeuCoReClass-AD provides a convenient function called run_experiment() (available in NeuCoReClassAD.py).
This function takes as input a dictionary of hyperparameters — including all the ones mentioned earlier, plus a seed for reproducibility — and automatically:

1. Instantiates the model

2. Trains it using the specified parameters

3. Evaluates its performance on the test set

👉 With a single call, you can run a complete experiment end-to-end.

In [9]:
from NeuCoReClassAD import * 

hyperparameters = {
        "dataset": "Epilepsy",
        "normality": 'sawing',
        "reverse": False,
        "n_transforms":12,
        "measure":"cosine",
        "temperature":0.1,
        "max_epochs": 0,
        "batch_size":32,
        "verbose":True,
        "seed": 123,
        }

run_experiment(hyperparameters)

Problem name:  epilepsy
Existing Classes:  ['epilepsy', 'walking', 'running', 'sawing']
Normal Class(es):  ['sawing']
Anomalous Class(es):  ['epilepsy' 'running' 'walking']
training time:  0.004962444305419922
inference time:  12.714001417160034
AUROC:  0.8919753086419753
AUPR:  0.9734353777122342


# Wrapping Up  🎉

With this notebook, you now have a complete guide to using NeuCoReClass-AD — from loading data and setting hyperparameters, to training, evaluating, and even running full experiments automatically. This framework is designed to be simple, flexible, and extensible, so you can easily adapt it to your own time series anomaly detection tasks.

Feel free to experiment with different datasets, tweak the hyperparameters, and explore how NeuCoReClass-AD performs in diverse scenarios :)